In [1]:
import pandas as pd
from numpy import nan
import numpy as np
from ast import literal_eval

In [2]:
# From: https://cernbox.cern.ch/files/link/public/Ju7DYsj0y8sQe2j?tiles-size=1&items-per-page=100&view-mode=resource-table
df = pd.read_csv("data/hww_simulated.csv")
df.drop(columns=["Particle.PT", "Particle.Eta", "Particle.Phi"], inplace=True)
df.head()

,Particle.PID,Electron.PT,Electron.Eta,Electron.Phi,Muon.PT,Muon.Eta,Muon.Phi,MissingET.MET,MissingET.Eta,MissingET.Phi
0,[ 21 21 25 24 13 -11 12 -14],[44.94672],[0.21702203],[1.4967911],[19.728607],[-0.76609546],[-0.8761875],[33.763885],[0.21087119],[-2.068956]
1,[ 21 21 25 -24 11 -13 14 -12],[32.669098],[0.28524885],[-0.93204],[19.569141],[0.722224],[1.1187549],[29.574873],[-0.76974607],[2.856485]
2,[ 21 21 25 -24 11 -13 14 -12],[],[],[],[],[],[],[0.],[0.],[0.]
3,[ 21 21 25 24 13 -11 12 -14],[],[],[],[],[],[],[52.082302],[2.2248452],[-1.8333149]
4,[ 21 21 25 -24 13 -11 12 -14],[],[],[],[25.512314],[-1.6375457],[1.6149265],[24.645163],[1.7733337],[-1.1305432]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Particle.PID   1000000 non-null  object
 1   Electron.PT    1000000 non-null  object
 2   Electron.Eta   1000000 non-null  object
 3   Electron.Phi   1000000 non-null  object
 4   Muon.PT        1000000 non-null  object
 5   Muon.Eta       1000000 non-null  object
 6   Muon.Phi       1000000 non-null  object
 7   MissingET.MET  1000000 non-null  object
 8   MissingET.Eta  1000000 non-null  object
 9   MissingET.Phi  1000000 non-null  object
dtypes: object(10)
memory usage: 76.3+ MB


In [4]:
# Interpret list of values as list, save 4 last values and explode into 4 columns; replace with type
df['Particle.PID'] = df['Particle.PID'].replace(r"  ", r", ", regex=True)
df['Particle.PID'] = df['Particle.PID'].replace(r" -", r", -", regex=True)
df['Particle.PID'] = df['Particle.PID'].replace(r"\[ ", r"[", regex=True)
df['Particle.PID'] = df['Particle.PID'].apply(literal_eval)
df['Particle.PID'] = df['Particle.PID'].apply(lambda x: x[-4:])

# Apply literal_eval to each of the first 9 columns except PID
for col in df.columns[1:]:
    df[col] = df[col].apply(literal_eval)
    df[col] = df[col].apply(lambda x: x[0] if len(x) > 0 else nan)
df

,Particle.PID,Electron.PT,Electron.Eta,Electron.Phi,Muon.PT,Muon.Eta,Muon.Phi,MissingET.MET,MissingET.Eta,MissingET.Phi
0,"[13, -11, 12, -14]",44.946720,0.217022,1.496791,19.728607,-0.766095,-0.876188,33.763885,0.210871,-2.068956
1,"[11, -13, 14, -12]",32.669098,0.285249,-0.932040,19.569141,0.722224,1.118755,29.574873,-0.769746,2.856485
2,"[11, -13, 14, -12]",NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
3,"[13, -11, 12, -14]",NaN,NaN,NaN,NaN,NaN,NaN,52.082302,2.224845,-1.833315
4,"[13, -11, 12, -14]",NaN,NaN,NaN,25.512314,-1.637546,1.614926,24.645163,1.773334,-1.130543
...,...,...,...,...,...,...,...,...,...,...
999995,"[13, -11, 12, -14]",11.255328,-0.749001,-1.809375,45.173160,0.503362,-0.187602,45.638530,-0.311425,2.709711
999996,"[13, -11, 12, -14]",18.383038,-2.296883,1.048567,33.396515,-1.980449,-0.525971,38.293423,2.399655,3.112945
999997,"[11, -13, 14, -12]",NaN,NaN,NaN,NaN,NaN,NaN,6.196441,-0.861511,1.593361
999998,"[11, -13, 14, -12]",27.799545,0.528331,-1.188118,NaN,NaN,NaN,27.849438,-0.526558,1.968728


In [5]:
from src.utils.change_of_coordinates import exp_to_four_vec

df[['Electron.E', 'Electron.px', 'Electron.py', 'Electron.pz']] = df.apply(
    lambda row: pd.Series(exp_to_four_vec(row['Electron.PT'], row['Electron.Eta'], row['Electron.Phi'], 0.000511)), axis=1)

df[['Muon.E', 'Muon.px', 'Muon.py', 'Muon.pz']] = df.apply(
    lambda row: pd.Series(exp_to_four_vec(row['Muon.PT'], row['Muon.Eta'], row['Muon.Phi'], 0.10566)), axis=1)

df.head()

,Particle.PID,Electron.PT,Electron.Eta,Electron.Phi,Muon.PT,Muon.Eta,Muon.Phi,MissingET.MET,MissingET.Eta,MissingET.Phi,Electron.E,Electron.px,Electron.py,Electron.pz,Muon.E,Muon.px,Muon.py,Muon.pz
0,"[13, -11, 12, -14]",44.946720,0.217022,1.496791,19.728607,-0.766095,-0.876188,33.763885,0.210871,-2.068956,46.009344,3.323257,44.823695,9.831179,25.806953,12.627984,-15.157571,-16.636398
1,"[11, -13, 14, -12]",32.669098,0.285249,-0.932040,19.569141,0.722224,1.118755,29.574873,-0.769746,2.856485,34.007226,19.477232,-26.227989,9.445712,24.898808,8.547856,17.603563,15.394421
2,"[11, -13, 14, -12]",NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[13, -11, 12, -14]",NaN,NaN,NaN,NaN,NaN,NaN,52.082302,2.224845,-1.833315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[13, -11, 12, -14]",NaN,NaN,NaN,25.512314,-1.637546,1.614926,24.645163,1.773334,-1.130543,NaN,NaN,NaN,NaN,68.079557,-1.125497,25.487476,-63.118434


In [6]:
def event_type(array, ids):
    """
    Determines event type by specifying all particles in a specific array must occur exactly once.
    :param array: array containing particle ids in event.
    :param ids: list containing id numbers of required particles.
    :returns event_type: Bool True if all ids in array exactly once.
    """
    event_type = False
    type_count = 0
    # Require all particles to appear once only
    for id in ids:
        if np.sum(array == id) == 1:
            type_count +=1
    # Require all particles to be present
    if type_count == len(ids):
        event_type = True
    return event_type

def get_event_type(array):
    ids_1 = [-13, 14, 11, -12]
    ids_2 = [13, -14, -11, 12]
    if event_type(np.array(array), ids_1):
        return 1
    elif event_type(np.array(array), ids_2):
        return 2
    return nan

df['Event.Type'] = df['Particle.PID'].apply(get_event_type)
df

,Particle.PID,Electron.PT,Electron.Eta,Electron.Phi,Muon.PT,Muon.Eta,Muon.Phi,MissingET.MET,MissingET.Eta,MissingET.Phi,Electron.E,Electron.px,Electron.py,Electron.pz,Muon.E,Muon.px,Muon.py,Muon.pz,Event.Type
0,"[13, -11, 12, -14]",44.946720,0.217022,1.496791,19.728607,-0.766095,-0.876188,33.763885,0.210871,-2.068956,46.009344,3.323257,44.823695,9.831179,25.806953,12.627984,-15.157571,-16.636398,2
1,"[11, -13, 14, -12]",32.669098,0.285249,-0.932040,19.569141,0.722224,1.118755,29.574873,-0.769746,2.856485,34.007226,19.477232,-26.227989,9.445712,24.898808,8.547856,17.603563,15.394421,1
2,"[11, -13, 14, -12]",NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,"[13, -11, 12, -14]",NaN,NaN,NaN,NaN,NaN,NaN,52.082302,2.224845,-1.833315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,"[13, -11, 12, -14]",NaN,NaN,NaN,25.512314,-1.637546,1.614926,24.645163,1.773334,-1.130543,NaN,NaN,NaN,NaN,68.079557,-1.125497,25.487476,-63.118434,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,"[13, -11, 12, -14]",11.255328,-0.749001,-1.809375,45.173160,0.503362,-0.187602,45.638530,-0.311425,2.709711,14.562848,-2.659878,-10.936519,-9.240894,51.017964,44.380568,-8.424939,23.710907,2
999996,"[13, -11, 12, -14]",18.383038,-2.296883,1.048567,33.396515,-1.980449,-0.525971,38.293423,2.399655,3.112945,92.316980,9.169709,15.932750,-90.468165,123.300064,28.882539,-16.766818,-118.691058,2
999997,"[11, -13, 14, -12]",NaN,NaN,NaN,NaN,NaN,NaN,6.196441,-0.861511,1.593361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
999998,"[11, -13, 14, -12]",27.799545,0.528331,-1.188118,NaN,NaN,NaN,27.849438,-0.526558,1.968728,31.770542,10.380531,-25.788743,15.380267,NaN,NaN,NaN,NaN,1


In [7]:
# l1 has highest p_T

df["p_l_1_E"] = df.apply(lambda row: row["Electron.E"] if row["Electron.PT"] > row["Muon.PT"] else row["Muon.E"], axis=1)
df["p_l_1_x"] = df.apply(lambda row: row["Electron.px"] if row["Electron.PT"] > row["Muon.PT"] else row["Muon.px"], axis=1)
df["p_l_1_y"] = df.apply(lambda row: row["Electron.py"] if row["Electron.PT"] > row["Muon.PT"] else row["Muon.py"], axis=1)
df["p_l_1_z"] = df.apply(lambda row: row["Electron.pz"] if row["Electron.PT"] > row["Muon.PT"] else row["Muon.pz"], axis=1)

df["p_l_2_E"] = df.apply(lambda row: row["Muon.E"] if row["Electron.PT"] > row["Muon.PT"] else row["Electron.E"], axis=1)
df["p_l_2_x"] = df.apply(lambda row: row["Muon.px"] if row["Electron.PT"] > row["Muon.PT"] else row["Electron.px"], axis=1)
df["p_l_2_y"] = df.apply(lambda row: row["Muon.py"] if row["Electron.PT"] > row["Muon.PT"] else row["Electron.py"], axis=1)
df["p_l_2_z"] = df.apply(lambda row: row["Muon.pz"] if row["Electron.PT"] > row["Muon.PT"] else row["Electron.pz"], axis=1)

df.drop(columns=["Electron.E", "Electron.px", "Electron.py", "Electron.pz", "Muon.E", "Muon.px", "Muon.py", "Muon.pz", "Particle.PID", "Electron.PT", "Electron.Eta", "Electron.Phi", "Muon.PT", "Muon.Eta", "Muon.Phi"], inplace=True)
df.head()

,MissingET.MET,MissingET.Eta,MissingET.Phi,Event.Type,p_l_1_E,p_l_1_x,p_l_1_y,p_l_1_z,p_l_2_E,p_l_2_x,p_l_2_y,p_l_2_z
0,33.763885,0.210871,-2.068956,2,46.009344,3.323257,44.823695,9.831179,25.806953,12.627984,-15.157571,-16.636398
1,29.574873,-0.769746,2.856485,1,34.007226,19.477232,-26.227989,9.445712,24.898808,8.547856,17.603563,15.394421
2,0.000000,0.000000,0.000000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52.082302,2.224845,-1.833315,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24.645163,1.773334,-1.130543,2,68.079557,-1.125497,25.487476,-63.118434,NaN,NaN,NaN,NaN


In [8]:
df["mpx"] = df.apply(lambda row: row["MissingET.MET"] * np.cos(row["MissingET.Phi"]), axis=1)
df["mpy"] = df.apply(lambda row: row["MissingET.MET"] * np.sin(row["MissingET.Phi"]), axis=1)

In [9]:
df.drop(columns=["MissingET.MET", "MissingET.Phi", "MissingET.Eta"], inplace=True)

In [10]:
df

,Event.Type,p_l_1_E,p_l_1_x,p_l_1_y,p_l_1_z,p_l_2_E,p_l_2_x,p_l_2_y,p_l_2_z,mpx,mpy
0,2,46.009344,3.323257,44.823695,9.831179,25.806953,12.627984,-15.157571,-16.636398,-16.132711,-29.660336
1,1,34.007226,19.477232,-26.227989,9.445712,24.898808,8.547856,17.603563,15.394421,-28.380976,8.318251
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-13.516069,-50.297933
4,2,68.079557,-1.125497,25.487476,-63.118434,NaN,NaN,NaN,NaN,10.502991,-22.295094
...,...,...,...,...,...,...,...,...,...,...,...
999995,2,51.017964,44.380568,-8.424939,23.710907,14.562848,-2.659878,-10.936519,-9.240894,-41.447994,19.103383
999996,2,123.300064,28.882539,-16.766818,-118.691058,92.316980,9.169709,15.932750,-90.468165,-38.277710,1.096874
999997,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.139809,6.194863
999998,1,NaN,NaN,NaN,NaN,31.770542,10.380531,-25.788743,15.380267,-10.792007,25.673406


In [11]:
# From https://cernbox.cern.ch/files/link/public/Ju7DYsj0y8sQe2j?tiles-size=1&items-per-page=100&view-mode=resource-table
# Extracted csv from output_tlvs.root
df_truth = pd.read_csv("data/df_final.csv")

df["Neutrino_1.E"] = df_truth["v1_E"]
df["Neutrino_1.x"] = df_truth["v1_p_x"]
df["Neutrino_1.y"] = df_truth["v1_p_y"]
df["Neutrino_1.z"] = df_truth["v1_p_z"]
df["Neutrino_2.E"] = df_truth["v2_E"]
df["Neutrino_2.x"] = df_truth["v2_p_x"]
df["Neutrino_2.y"] = df_truth["v2_p_y"]
df["Neutrino_2.z"] = df_truth["v2_p_z"]

To infer which lepton does each neutrino associated with, we can consider 2 combinations

A: $(l_1,v_1)$ and $(l_2,v_2)$

B: $(l_1,v_2)$ and $(l_2,v_1)$

For each variant we can compute mass of reconstruced W boson and compare with to known physical mass of 80.4. Therefore for each combination following metric is calculated:
$$ \varDelta = |M_{W1} - 80.4| + |M_{W2} - 80.4| ,$$
where $M_{Wi} = \sqrt{(E_1 + E_2)^2 + (p_{x1} + p_{x2})^2 + (p_{y1} + p_{y2})^2 + (p_{z1} + p_{z2})^2}$ . Indexes refer to the first and second particle in the lepton-neutroni pair.

Then we select combination with lesser delta.

In [12]:
def get_mass (E1, x1, y1, z1, E2, x2, y2, z2):
    return np.sqrt((E1 + E2)**2 + (x1 + x2)**2 + (y1 + y2)**2 + (z1 + z2)**2)

W_mass = 80.4

# A
A_mass1 = get_mass(df['p_l_1_E'],      df['p_l_1_x'],      df['p_l_1_y'],      df['p_l_1_z'],
                   df['Neutrino_1.E'], df['Neutrino_1.x'], df['Neutrino_1.y'], df['Neutrino_1.z'])
A_mass2 = get_mass(df['p_l_2_E'],      df['p_l_2_x'],      df['p_l_2_y'],      df['p_l_2_z'],
                   df['Neutrino_2.E'], df['Neutrino_2.x'], df['Neutrino_2.y'], df['Neutrino_2.z'])
A_mass1

0          85.284991
1         179.581148
2                NaN
3                NaN
4         826.097914
             ...    
999995    108.790365
999996    782.240729
999997           NaN
999998           NaN
999999           NaN
Length: 1000000, dtype: float64

In [13]:
df['Delta1'] = np.abs(A_mass1 - W_mass) + np.abs(A_mass2 - W_mass)

# B
B_mass1 = get_mass(df['p_l_1_E'],      df['p_l_1_x'],      df['p_l_1_y'],      df['p_l_1_z'],
                   df['Neutrino_2.E'], df['Neutrino_2.x'], df['Neutrino_2.y'], df['Neutrino_2.z'])
B_mass2 = get_mass(df['p_l_2_E'],      df['p_l_2_x'],      df['p_l_2_y'],      df['p_l_2_z'],
                   df['Neutrino_1.E'], df['Neutrino_1.x'], df['Neutrino_1.y'], df['Neutrino_1.z'])
B_mass1

0          79.879014
1          69.189795
2                NaN
3                NaN
4         286.736013
             ...    
999995     97.043225
999996    380.547240
999997           NaN
999998           NaN
999999           NaN
Length: 1000000, dtype: float64

In [14]:
df['Delta2'] = np.abs(B_mass1 - W_mass) + np.abs(B_mass2 - W_mass)
df

,Event.Type,p_l_1_E,p_l_1_x,p_l_1_y,p_l_1_z,p_l_2_E,p_l_2_x,p_l_2_y,p_l_2_z,mpx,...,Neutrino_1.E,Neutrino_1.x,Neutrino_1.y,Neutrino_1.z,Neutrino_2.E,Neutrino_2.x,Neutrino_2.y,Neutrino_2.z,Delta1,Delta2
0,2,46.009344,3.323257,44.823695,9.831179,25.806953,12.627984,-15.157571,-16.636398,-16.132711,...,24.155747,-9.751504,1.652740,-22.038074,32.633243,-5.192401,-31.834253,-4.954573,6.895285,16.247646
1,1,34.007226,19.477232,-26.227989,9.445712,24.898808,8.547856,17.603563,15.394421,-28.380976,...,104.635343,-33.046353,-6.449344,99.070175,23.813755,6.082861,15.858921,16.690967,110.761226,105.755433
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,454.164621,10.101091,36.836577,-452.555563,44.485661,-1.417896,6.964154,-43.914282,NaN,NaN
3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-13.516069,...,11.520140,0.308590,-9.489610,-6.524239,209.116425,-13.208733,-42.939242,-204.233764,NaN,NaN
4,2,68.079557,-1.125497,25.487476,-63.118434,NaN,NaN,NaN,NaN,10.502991,...,518.828622,6.080513,-24.439560,-518.217014,136.289820,3.973394,1.038273,-136.227931,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2,51.017964,44.380568,-8.424939,23.710907,14.562848,-2.659878,-10.936519,-9.240894,-41.447994,...,43.795926,-23.368973,31.438789,19.585115,28.627128,-16.838359,-11.530906,20.075367,55.239210,29.141874
999996,2,123.300064,28.882539,-16.766818,-118.691058,92.316980,9.169709,15.932750,-90.468165,-38.277710,...,432.475998,-18.686029,18.436389,-431.678609,148.170772,-20.691191,-16.035348,-145.840049,958.796159,960.906279
999997,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.139809,...,149.923697,-4.130795,15.310890,-149.082622,315.802773,2.631361,-7.949812,-315.691730,NaN,NaN
999998,1,NaN,NaN,NaN,NaN,31.770542,10.380531,-25.788743,15.380267,-10.792007,...,29.060285,16.264045,9.606509,22.083840,42.231540,-12.737246,27.376456,29.526179,NaN,NaN


In [15]:
df = df.dropna(how='any').copy()
df

,Event.Type,p_l_1_E,p_l_1_x,p_l_1_y,p_l_1_z,p_l_2_E,p_l_2_x,p_l_2_y,p_l_2_z,mpx,...,Neutrino_1.E,Neutrino_1.x,Neutrino_1.y,Neutrino_1.z,Neutrino_2.E,Neutrino_2.x,Neutrino_2.y,Neutrino_2.z,Delta1,Delta2
0,2,46.009344,3.323257,44.823695,9.831179,25.806953,12.627984,-15.157571,-16.636398,-16.132711,...,24.155747,-9.751504,1.652740,-22.038074,32.633243,-5.192401,-31.834253,-4.954573,6.895285,16.247646
1,1,34.007226,19.477232,-26.227989,9.445712,24.898808,8.547856,17.603563,15.394421,-28.380976,...,104.635343,-33.046353,-6.449344,99.070175,23.813755,6.082861,15.858921,16.690967,110.761226,105.755433
9,2,49.597160,22.813978,15.762331,-41.121158,69.832646,-17.100484,-6.723840,-67.371735,-5.826737,...,26.862526,13.568716,-23.093071,2.048255,29.344590,-18.987219,14.320044,-17.190893,68.193636,62.414710
12,1,175.467569,21.486192,-19.338825,-173.069957,45.657054,13.068566,-1.341534,-43.726187,-34.443836,...,166.030231,-14.434660,8.715525,-165.171783,1017.878899,-17.337016,11.145277,-1017.670214,1822.615929,1822.529619
14,1,35.998664,-18.668192,-28.457580,-11.728484,24.345485,3.566330,-24.006044,-1.921922,14.639708,...,33.715197,-8.949950,29.926252,-12.689853,33.197795,24.752230,21.648865,4.554922,17.694146,30.232552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999990,1,41.619752,-32.964294,12.966198,-21.850072,15.758931,-9.596379,-4.901875,11.498915,42.427841,...,45.971416,39.393362,9.305436,-21.793188,36.180151,2.918932,-16.748577,-31.936943,40.331116,29.858115
999991,1,102.809805,-38.071159,-27.614804,-91.421362,45.572115,16.044550,14.255055,-40.201645,22.012483,...,16.674753,12.046663,5.232006,10.273822,20.443327,10.176135,8.112032,-15.766129,80.814974,89.783919
999992,2,54.509415,-33.074680,17.878401,39.467625,42.182816,-20.680468,-13.944449,34.018532,53.682804,...,78.759600,38.007458,-13.590390,67.629942,49.366564,16.355877,10.789491,45.311475,131.550969,138.627612
999995,2,51.017964,44.380568,-8.424939,23.710907,14.562848,-2.659878,-10.936519,-9.240894,-41.447994,...,43.795926,-23.368973,31.438789,19.585115,28.627128,-16.838359,-11.530906,20.075367,55.239210,29.141874


In [16]:
condition = df["Delta1"] < df["Delta2"]

df["p_v_1_E"] = df["Neutrino_1.E"].where(condition, df["Neutrino_2.E"])
df["p_v_1_x"] = df["Neutrino_1.x"].where(condition, df["Neutrino_2.x"])
df["p_v_1_y"] = df["Neutrino_1.y"].where(condition, df["Neutrino_2.y"])
df["p_v_1_z"] = df["Neutrino_1.z"].where(condition, df["Neutrino_2.z"])
df["p_v_2_E"] = df["Neutrino_2.E"].where(condition, df["Neutrino_1.E"])
df["p_v_2_x"] = df["Neutrino_2.x"].where(condition, df["Neutrino_1.x"])
df["p_v_2_y"] = df["Neutrino_2.y"].where(condition, df["Neutrino_1.y"])
df["p_v_2_z"] = df["Neutrino_2.z"].where(condition, df["Neutrino_1.z"])

In [17]:
df

,Event.Type,p_l_1_E,p_l_1_x,p_l_1_y,p_l_1_z,p_l_2_E,p_l_2_x,p_l_2_y,p_l_2_z,mpx,...,Delta1,Delta2,p_v_1_E,p_v_1_x,p_v_1_y,p_v_1_z,p_v_2_E,p_v_2_x,p_v_2_y,p_v_2_z
0,2,46.009344,3.323257,44.823695,9.831179,25.806953,12.627984,-15.157571,-16.636398,-16.132711,...,6.895285,16.247646,24.155747,-9.751504,1.652740,-22.038074,32.633243,-5.192401,-31.834253,-4.954573
1,1,34.007226,19.477232,-26.227989,9.445712,24.898808,8.547856,17.603563,15.394421,-28.380976,...,110.761226,105.755433,23.813755,6.082861,15.858921,16.690967,104.635343,-33.046353,-6.449344,99.070175
9,2,49.597160,22.813978,15.762331,-41.121158,69.832646,-17.100484,-6.723840,-67.371735,-5.826737,...,68.193636,62.414710,29.344590,-18.987219,14.320044,-17.190893,26.862526,13.568716,-23.093071,2.048255
12,1,175.467569,21.486192,-19.338825,-173.069957,45.657054,13.068566,-1.341534,-43.726187,-34.443836,...,1822.615929,1822.529619,1017.878899,-17.337016,11.145277,-1017.670214,166.030231,-14.434660,8.715525,-165.171783
14,1,35.998664,-18.668192,-28.457580,-11.728484,24.345485,3.566330,-24.006044,-1.921922,14.639708,...,17.694146,30.232552,33.715197,-8.949950,29.926252,-12.689853,33.197795,24.752230,21.648865,4.554922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999990,1,41.619752,-32.964294,12.966198,-21.850072,15.758931,-9.596379,-4.901875,11.498915,42.427841,...,40.331116,29.858115,36.180151,2.918932,-16.748577,-31.936943,45.971416,39.393362,9.305436,-21.793188
999991,1,102.809805,-38.071159,-27.614804,-91.421362,45.572115,16.044550,14.255055,-40.201645,22.012483,...,80.814974,89.783919,16.674753,12.046663,5.232006,10.273822,20.443327,10.176135,8.112032,-15.766129
999992,2,54.509415,-33.074680,17.878401,39.467625,42.182816,-20.680468,-13.944449,34.018532,53.682804,...,131.550969,138.627612,78.759600,38.007458,-13.590390,67.629942,49.366564,16.355877,10.789491,45.311475
999995,2,51.017964,44.380568,-8.424939,23.710907,14.562848,-2.659878,-10.936519,-9.240894,-41.447994,...,55.239210,29.141874,28.627128,-16.838359,-11.530906,20.075367,43.795926,-23.368973,31.438789,19.585115


In [18]:
df = df.drop(columns=['Neutrino_1.E', 'Neutrino_1.x', 'Neutrino_1.y', 'Neutrino_1.z', 'Neutrino_2.E', 'Neutrino_2.x', 'Neutrino_2.y', 'Neutrino_2.z', 'Delta1', 'Delta2'])
df

,Event.Type,p_l_1_E,p_l_1_x,p_l_1_y,p_l_1_z,p_l_2_E,p_l_2_x,p_l_2_y,p_l_2_z,mpx,mpy,p_v_1_E,p_v_1_x,p_v_1_y,p_v_1_z,p_v_2_E,p_v_2_x,p_v_2_y,p_v_2_z
0,2,46.009344,3.323257,44.823695,9.831179,25.806953,12.627984,-15.157571,-16.636398,-16.132711,-29.660336,24.155747,-9.751504,1.652740,-22.038074,32.633243,-5.192401,-31.834253,-4.954573
1,1,34.007226,19.477232,-26.227989,9.445712,24.898808,8.547856,17.603563,15.394421,-28.380976,8.318251,23.813755,6.082861,15.858921,16.690967,104.635343,-33.046353,-6.449344,99.070175
9,2,49.597160,22.813978,15.762331,-41.121158,69.832646,-17.100484,-6.723840,-67.371735,-5.826737,-8.669281,29.344590,-18.987219,14.320044,-17.190893,26.862526,13.568716,-23.093071,2.048255
12,1,175.467569,21.486192,-19.338825,-173.069957,45.657054,13.068566,-1.341534,-43.726187,-34.443836,20.310764,1017.878899,-17.337016,11.145277,-1017.670214,166.030231,-14.434660,8.715525,-165.171783
14,1,35.998664,-18.668192,-28.457580,-11.728484,24.345485,3.566330,-24.006044,-1.921922,14.639708,52.443168,33.715197,-8.949950,29.926252,-12.689853,33.197795,24.752230,21.648865,4.554922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999990,1,41.619752,-32.964294,12.966198,-21.850072,15.758931,-9.596379,-4.901875,11.498915,42.427841,-7.870438,36.180151,2.918932,-16.748577,-31.936943,45.971416,39.393362,9.305436,-21.793188
999991,1,102.809805,-38.071159,-27.614804,-91.421362,45.572115,16.044550,14.255055,-40.201645,22.012483,13.572818,16.674753,12.046663,5.232006,10.273822,20.443327,10.176135,8.112032,-15.766129
999992,2,54.509415,-33.074680,17.878401,39.467625,42.182816,-20.680468,-13.944449,34.018532,53.682804,-4.234704,78.759600,38.007458,-13.590390,67.629942,49.366564,16.355877,10.789491,45.311475
999995,2,51.017964,44.380568,-8.424939,23.710907,14.562848,-2.659878,-10.936519,-9.240894,-41.447994,19.103383,28.627128,-16.838359,-11.530906,20.075367,43.795926,-23.368973,31.438789,19.585115


In [19]:
df.to_csv("data/hww_simulated_final.csv", index=False)